### Genetic Algorithm 
- uses energy efficiency dataset https://archive.ics.uci.edu/ml/datasets/energy+efficiency#


We perform energy analysis using 12 different building shapes simulated in Ecotect. The buildings differ with respect to the glazing area, the glazing area distribution, and the orientation, amongst other parameters. We simulate various settings as functions of the afore-mentioned characteristics to obtain 768 building shapes. The dataset comprises 768 samples and 8 features, aiming to predict two real valued responses. It can also be used as a multi-class classification problem if the response is rounded to the nearest integer.


Attribute Information:

The dataset contains eight attributes (or features, denoted by X1...X8) and two responses (or outcomes, denoted by y1 and y2). The aim is to use the eight features to predict each of the two responses. 

Specifically: 
- X1	Relative Compactness 
- X2	Surface Area 
- X3	Wall Area 
- X4	Roof Area 
- X5	Overall Height 
- X6	Orientation 
- X7	Glazing Area 
- X8	Glazing Area Distribution 
- y1	Heating Load 
- y2	Cooling Load



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn import svm

/home/tdarrah/anaconda3/envs/auto36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Get the dataset
- df.sample(frac=1) shuffles the data 

In [2]:
df = pd.read_csv("energy_efficiency.csv")
df = df.sample(frac=1)
rows = len(df)
cols = len(df.keys())
print(rows, cols)
df.head()

768 10


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
676,0.90,563.5,318.5,122.50,7.0,2,0.40,4,36.45,33.98
312,0.74,686.0,245.0,220.50,3.5,2,0.25,1,12.43,15.19
626,0.98,514.5,294.0,110.25,7.0,4,0.40,3,32.23,33.86
409,0.74,686.0,245.0,220.50,3.5,3,0.25,3,12.03,13.79
689,0.79,637.0,343.0,147.00,7.0,3,0.40,4,41.73,47.59


### X6 and X8 are categorical variables

In [3]:
X = pd.get_dummies(df.iloc[:,0:(cols-2)], columns=['X6', 'X8']).values
y = df.iloc[:, (cols-2):(cols-1)].values
print(X[0])
print(y[0])

[9.000e-01 5.635e+02 3.185e+02 1.225e+02 7.000e+00 4.000e-01 1.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 1.000e+00 0.000e+00]
[36.45]


### Scale the data with minmax scaler

In [4]:
mms = preprocessing.MinMaxScaler()
X = mms.fit_transform(X)
print(X[0])

[0.77777778 0.16666667 0.42857143 0.11111111 1.         1.
 1.         0.         0.         0.         0.         0.
 0.         0.         1.         0.        ]


### Set genetic algorithm parameters
- M : number of generations
- N : population size
- Pc: probability of crossover
- Pm: probability of mutation
- l : string size
- k : tournament selection contestants

### **since we are performing a GA for SVM, the parameters to tune are c and gamma**

#### SVM hyperparameters
- kernel: poly, rbf, linear
- poly: degree of polynomial
- c: controls the tradeoff between low error and maximizing the norm of the weight
- gamma: determines strength of training sample with gaussian kernel

In [5]:
#probability of crossover
Pc = .9 #these do not need to sum to 1
#probability of mutation
Pm = .1
#size of population
pop = 100
#number of generations
gen = 400
l = 24 # first 12 are for c, second 12 are for gamma

In [6]:
# the chromosome
xy = np.random.choice([0,1], size=(l,), p=[.5, .5])
print(xy)

[1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 0]


### (1) Create a random population of chromosomes (potential solutions)

In [7]:
population = np.empty((0, len(xy)))
for i in range(gen):
    random.shuffle(xy)
    population = np.vstack((population, xy))

In [8]:
print(population[0:5])

[[0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.]]


### (2) Calculate precision of the chromosomes
- Range (a,b)
- length, l
- precision (b-a)/(2^l-1)
- encode (literal base 2 encoding)
- decode (sum(bit*2^i)*precision + a

In [9]:
# parameter c, see precision formula
ac = 10
bc = 1000
lc = len(xy)/2
pc = (bc - ac)/((2**lc)-1)

In [10]:
# parameter gamma, see precision formula
ag = .05
bg = .99
lg = len(xy)/2
pg = (bg - ag)/((2**lg)-1)

### Decode the chromosomes
- returns a real value in the specified range for the parameter

In [11]:
def decode(xy, index, precision, lowerBound):
    """Decodes a chromosome into a real value"""
    power = 0 #binary powers
    sum = 0
    for i in range(len(xy)//2):
        val = xy[index]*(2**power)
        sum += val
        index -= 1
        power += 1
    return (sum * precision) + lowerBound

In [12]:
#index of the start of the c parameter from SVM 
cIndex = -1 # start at the end of the chromosome
#index of the start of the gamma parameter from SVM
gIndex = (l//2) + 1 #start in the middle of the chromosome
c1 = decode(xy, cIndex, pc, ac)
g1 = decode(xy, gIndex, pg, ag)
print("c bounds: ({},{})\tcvalue: {}".format(ac,bc,c1))
print("g bounds: ({},{})\tgvalue: {}".format(ag,bg,g1))


c bounds: (10,1000)	cvalue: 54.72527472527472
g bounds: (0.05,0.99)	gvalue: 0.19874725274725275


### Perform the algorithm

- keep track of children and mutations 

In [18]:
def parentSelection(population, l):
    """
    returns two parents from a population given the chromosome size l
    """
    parents = np.zeros((0, l))
    for k in range(2):
        #get some random samples to pick good parents
        candidates = population[np.random.choice(len(population), 3, replace=False)]

        #the best selection for a parent
        selection = -1
        score = 10000000

        for choice in candidates:
            #c and gamma values for SVR
            c = decode(choice, cIndex, pc, ac)
            g = decode(choice, gIndex, pg, ag)

            total = 0
            kf = cross_validation.KFold(len(X),n_folds=num_folds)

            for train_index, test_index in kf:
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                model = svm.SVR(kernel='rbf',C=c, gamma=g)
                model.fit(X_train, y_train)
                acc = model.score(X_test, y_test)
                err = 1-acc
                total += err

            total = total/num_folds
            print(total)
            if (total < score):
                score = total
                selection = choice

        print(selection, score)
        parents = np.vstack((parents, selection))
    return parents

In [14]:
bestC = []
worstC = []
bestG = []
worstC = []

ofg = np.empty((0, len(xy)+2))
ofgf = []

mgm1 = np.empty((0, len(xy)+1))
mgm2 = np.empty((0, len(xy)+1))

mgm11 = np.empty((0, len(xy)+2))
mgm22 = np.empty((0, len(xy)+2))

mgm111 = np.empty((0, len(xy)+2))
mgm222 = np.empty((0, len(xy)+2))

In [19]:
#index of the start of the c parameter from SVM 
cIndex = -1 # start at the end of the chromosome
#index of the start of the gamma parameter from SVM
gIndex = (l//2) + 1 #start in the middle of the chromosome

for i in range(1):
    newPopulation = np.empty((0, len(xy)))
    
    agc1 = np.empty((0, len(xy)+1))
    acg2 = np.empty((0, len(xy)+1))
    
    mgc1 = []
    mgc2 = []
    
    bestgc = np.empty((0, len(xy)+1))
    fbgc = []
    fwgc = []
    
    num_folds = 3
    
    for j in range(1): #pop//2
        #dads = population[np.random.choice(len(population), 3, replace=False)]
        parents = parentSelection(population, l)
        print(parents)
        

        

0.1440026226600817
0.09681450560926481
0.09989113947893002
[0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.] 0.09681450560926481
0.08118092381313806
0.07194373618504753
0.06349676317621673
[0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.] 0.06349676317621673
[[0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.]]
